In [13]:
import requests
import json
import nltk
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
pd.set_option("display.max_colwidth", None)

/home/vija1705/.pyenv/versions/3.8.6/envs/transformers-test/lib/python3.8/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [36]:
train_df = pd.read_csv("../data/training_df.csv", keep_default_na=False)
val_df = pd.read_csv("../data/valid_df.csv", keep_default_na=False)

In [35]:
train_df.count()

arg_id          20635
key_point_id    20635
label           20635
argument        20635
topic           20635
stance          20635
key_point       20635
dtype: int64

In [33]:
labels_test.head()

,key_point_id,key_point,topic,stance
0,kp_0_0,"Routine child vaccinations, or their side effects, are dangerous",Routine child vaccinations should be mandatory,-1
1,kp_0_1,Mandatory vaccination contradicts basic rights,Routine child vaccinations should be mandatory,-1
2,kp_0_2,The parents and not the state should decide,Routine child vaccinations should be mandatory,-1
3,kp_0_3,Routine child vaccinations are not necessary to keep children healthy,Routine child vaccinations should be mandatory,-1
4,kp_0_4,Routine child vaccinations are effective,Routine child vaccinations should be mandatory,1


In [8]:
labels_test.count()

key_point_id    33
key_point       33
topic           33
stance          33
dtype: int64

In [10]:
def sentence_segmentation_dict(text):
    sentences = nltk.sent_tokenize(text)
    return {idx:sent for idx, sent in enumerate(sentences, start=1)}

In [4]:
userID = "UV4HkxoP"
apiKey = "CAg9J7zNCfCr+6yhVOSvUARf8R9KuxkP"

In [5]:
def identify_aspects(text, topic):
    arguments= sentence_segmentation_dict(text)
    payload = {
        "query": topic,
        "arguments": arguments,
        "userID": userID,
        "apiKey": apiKey
    }
    is_timed_out = True
    timed_out_ctr = 5
    while is_timed_out == True:
        try:
            json_dict = requests.post("https://api.argumentsearch.com/en/get_aspects", timeout=1000, data=json.dumps(payload),
                                      headers={'Content-Type': 'application/json'}).json()
            is_timed_out = False
        except requests.exceptions.Timeout or ConnectionError or json.decoder.JSONDecodeError:
            print("Timed out for {0} times.".format(str(timed_out_ctr)))
        except Exception as e:
            print(e)
    return json_dict

In [39]:
def aspect_identification(row):
    topic = row['topic']
    argument = row['argument']
    response_dict = identify_aspects(argument, topic)
    aspects = response_dict["aspects"] if "aspects" in response_dict else []
    row['aspects'] = aspects if len(aspects) else None
    return row

In [20]:
tmp_df = args_test

In [40]:
tmp_df = tmp_df.progress_apply(aspect_identification, axis=1)

In [30]:
tmp_df.count()

arg_id      723
argument    723
topic       723
stance      723
aspects     723
dtype: int64

In [ ]:
tmp_df

In [18]:
tmp_df.head(10)

,arg_id,argument,topic,stance,aspects
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,"[spread, child vaccinations, mandatory]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,"[child vaccinations, side effects]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,"[child vaccinations, catch]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[side effects]
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[free, mandatory, side effects]"
5,arg_0_5,"Child vaccination is not necessary, and should not be mandatory, because vaccines are a lie. Let them decide if they want to be vaccinated.",Routine child vaccinations should be mandatory,-1,"[child vaccination, vaccinated]"
6,arg_0_6,Child vaccination isn't necessary and may result in other diseases for children,Routine child vaccinations should be mandatory,-1,"[child vaccination, diseases]"
7,arg_0_7,Child vaccination should not be mandatory as there are often side effects from them,Routine child vaccinations should be mandatory,-1,"[child vaccination, side effects]"
8,arg_0_8,Child vaccination should not be mandatory because they don't suffer from symptoms,Routine child vaccinations should be mandatory,-1,"[child vaccination, mandatory]"
9,arg_0_9,"Child vaccination should not be mandatory, it should be the parents' decision, since the adverse effects that the vaccine can cause to the infant are not known",Routine child vaccinations should be mandatory,-1,"[adverse effects, child vaccination]"


In [38]:
tmp_df.count()

arg_id      723
argument    723
topic       723
stance      723
aspects     723
dtype: int64

In [41]:
tmp_df.to_csv("../data/sample_aspects.csv", index=False)

In [ ]:
all_df = labels_test.merge(arg_df, how='inner', left_on='arg_id', right_on='arg_id')
all_df = all_df.merge(kp_df[['key_point_id', 'key_point']], how='inner', left_on='key_point_id', right_on='key_point_id')